The purpose of these experiments is to evaluate the predictive performance **(test accuracy)** of MF as a function of 

(i) fraction of training data and 

(ii) training time.

- We divide the training data into **100 mini-batches** and we compare the performance of online random forests (MF, ORF-Saffari [20]) to batch random forests (Breiman-RF, ERT-k, ERT-1) which are trained on the same fraction of the training data.
- We evaluate on four of the five datasets (usps, satimages, letter, dna) — we excluded the mushroom dataset as even very simple logical rules achieve > 99% accuracy on this dataset. 
- We re-scaled the datasets such that each feature takes on values in the range [0, 1] (by subtracting the min value along that dimension and dividing by the range along that dimension, where range = max − min).

As is common in the random forest literature [2], we set **the number of trees M = 100.**

- For Mondrian forests, we set the lifetime λ = ∞ and the HNSP discount parameter γ = 10D. 

- For Breiman-RF and ERT, the hyper parameters are set to default values. 

- We repeat each algorithm with five random initializations and report the mean performance.

# Load data

In [1]:
import h5py
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file

In [8]:
def load_usps():
    with h5py.File('data/usps.h5', 'r') as hf:
        train = hf.get('train')
        test = hf.get('test')
        
    X_train = train.get('data')[:]
    y_train = train.get('target')[:]
    X_test = test.get('data')[:]
    y_test = test.get('target')[:]

    print('Loading usps...\n')
    print('X_train.shape:\t', X_train.shape)
    print('y_train.shape:\t', y_train.shape)
    print('X_test.shape:\t', X_test.shape)
    print('y_test.shape:\t', y_test.shape)
    return X_train, X_test, y_train, y_test


def load_letter():
    dataset = pd.read_csv('data/letter-recognition.data', header=None)

    data = dataset.loc[:,1:]
    data = np.array(data)

    target = dataset[0]
    target = target.apply(lambda x: ord(x) - ord('A'))
    target = np.array(target)

    X_train, X_test, y_train, y_test = train_test_split(data, target)

    print('Loading letter...\n')
    print('X_train.shape:\t', X_train.shape)
    print('y_train.shape:\t', y_train.shape)
    print('X_test.shape:\t', X_test.shape)
    print('y_test.shape:\t', y_test.shape)
    return X_train, X_test, y_train, y_test


def load_satim():
    train = pd.read_csv('data/sat.trn', sep=' ', header=None)
    test = pd.read_csv('data/sat.tst', sep=' ', header=None)
    
    train = np.array(train)
    X_train = train[:,:36]
    y_train = train[:,36]

    test = np.array(test)
    X_test = test[:,:36]
    y_test = test[:,36]

    print('Loading satim...\n')
    print('X_train.shape:\t', X_train.shape)
    print('y_train.shape:\t', y_train.shape)
    print('X_test.shape:\t', X_test.shape)
    print('y_test.shape:\t', y_test.shape)
    return X_train, X_test, y_train, y_test


def load_dna():
    data, target = load_svmlight_file('data/dna.scale.txt')
    X_train, X_test, y_train, y_test = train_test_split(data, target)
    X_train = X_train.A
    X_test = X_test.A
    print('Loading dna...\n')
    print('X_train.shape:\t', X_train.shape)
    print('y_train.shape:\t', y_train.shape)
    print('X_test.shape:\t', X_test.shape)
    print('y_test.shape:\t', y_test.shape)
    return X_train, X_test, y_train, y_test


def iterate_batches(X_train, y_train, n_batches=100):
    n_samples = X_train.shape[0]
    batch_size = n_samples // 100
    
    for i in range(n_batches):
        start = i * batch_size
        end = (i + 1) * batch_size
        yield X_train[start:end], y_train[start:end]

In [9]:
X_train, X_test, y_train, y_test = load_usps()
X_train, X_test, y_train, y_test = load_letter()
X_train, X_test, y_train, y_test = load_satim()
X_train, X_test, y_train, y_test = load_dna()

Loading satim...

X_train.shape:	 (4435, 36)
y_train.shape:	 (4435,)
X_test.shape:	 (2000, 36)
y_test.shape:	 (2000,)


# Classical RF

Compare (both in time complexity & performance) the coded algorithm [on 2-3 real-world datasets] with classical random forests in the online mode, i.e. with forests that are

(1) completely refitted on data of steps [1, t] each step t

(2) Partially fitted for every new observation (some new trees fitted, some old removed)

(3) rolling-window refitting on [t-h, t]

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
n_batches = 100